# PaddleOCR Text Recognition Training on Amazon SageMaker

This notebook demonstrates how to train **Text Recognition models only** using PaddleOCR on Amazon SageMaker with GPU support.

## Key Features:
- **เฉพาะ Text Recognition** (ไม่รวม Detection)
- ใช้ `tools/train_rec.py` script
- รูปแบบ annotation: `image_path\ttext_content`
- รองรับ CRNN, SVTR, PP-OCRv4 architectures
- S3 integration สำหรับ data และ model management

## Requirements:
- Amazon SageMaker Notebook Instance ที่รองรับ GPU
- PaddlePaddle GPU version
- การเข้าถึง S3 bucket สำหรับเก็บข้อมูลและ models

## 1. Environment Setup & GPU Check

ตรวจสอบสภาพแวดล้อมและติดตั้ง dependencies ที่จำเป็นสำหรับการเทรน Text Recognition

In [ ]:
import sys
import os
import subprocess
import json
import boto3
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import yaml
from pathlib import Path
import time
from datetime import datetime

# ตรวจสอบ Python version
print(f"Python version: {sys.version}")

# ตรวจสอบ SageMaker version
print(f"SageMaker version: {sagemaker.__version__}")

# ตั้งค่า SageMaker session และ role
sagemaker_session = sagemaker.Session()
role = get_execution_role()
print(f"SageMaker Role: {role}")

# ตั้งค่า S3 bucket
bucket = sagemaker_session.default_bucket()
print(f"Default S3 bucket: {bucket}")

print("✅ Basic environment setup completed!")

In [ ]:
# ติดตั้ง PaddlePaddle GPU version
print("🚀 Installing PaddlePaddle GPU version...")
subprocess.run([
    sys.executable, "-m", "pip", "install", 
    "paddlepaddle-gpu==2.5.2", "-f", 
    "https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html"
], check=True)

# ติดตั้ง dependencies เพิ่มเติม
print("📦 Installing additional dependencies...")
subprocess.run([
    sys.executable, "-m", "pip", "install", 
    "opencv-python-headless", "imgaug", "pyclipper", "lmdb", "tqdm", 
    "numpy", "visualdl", "python-Levenshtein", "shapely", "pyclipper",
    "easydict", "scikit-image"
], check=True)

print("✅ Dependencies installed successfully!")

In [ ]:
# ตรวจสอบ GPU availability
import paddle

print(f"🔍 PaddlePaddle version: {paddle.__version__}")
print(f"🎮 GPU available: {paddle.is_compiled_with_cuda()}")

if paddle.is_compiled_with_cuda():
    print(f"🔥 GPU count: {paddle.device.cuda.device_count()}")
    # แสดงข้อมูล GPU
    for i in range(paddle.device.cuda.device_count()):
        gpu_name = paddle.device.cuda.get_device_properties(i).name
        gpu_memory = paddle.device.cuda.get_device_properties(i).total_memory / (1024**3)
        print(f"   GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")
else:
    print("❌ GPU not available! Please use GPU-enabled instance.")
    
# ตรวจสอบว่า GPU พร้อมใช้งาน
assert paddle.is_compiled_with_cuda(), "GPU is required for training!"
print("✅ GPU check passed!")

## 2. Clone PaddleOCR Repository & Install Dependencies

โคลน Official PaddleOCR Repository และติดตั้ง dependencies ที่จำเป็น

In [ ]:
# ตั้งค่า path สำหรับ PaddleOCR
PADDLEOCR_DIR = "/home/ec2-user/SageMaker/PaddleOCR"
WORK_DIR = "/home/ec2-user/SageMaker/paddle_ocr_recognition"

# สร้าง working directory
os.makedirs(WORK_DIR, exist_ok=True)

# โคลน PaddleOCR repository
if not os.path.exists(PADDLEOCR_DIR):
    print("🚀 Cloning PaddleOCR repository...")
    subprocess.run([
        "git", "clone", "https://github.com/PaddlePaddle/PaddleOCR.git", 
        PADDLEOCR_DIR
    ], check=True)
    print("✅ Repository cloned successfully!")
else:
    print("📁 PaddleOCR repository already exists.")

# เปลี่ยนไป working directory
os.chdir(PADDLEOCR_DIR)
print(f"📂 Current directory: {os.getcwd()}")

# ติดตั้ง PaddleOCR requirements
print("📦 Installing PaddleOCR requirements...")
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], check=True)

print("✅ PaddleOCR setup completed!")

In [ ]:
# ตั้งค่า S3 paths สำหรับ Recognition data
S3_BUCKET = bucket
S3_RECOGNITION_DATA_PREFIX = "paddleocr-recognition-data"
S3_RECOGNITION_MODEL_PREFIX = "paddleocr-recognition-models"

# Local paths สำหรับ Recognition
LOCAL_RECOGNITION_DATA_DIR = os.path.join(WORK_DIR, "recognition_data")
LOCAL_RECOGNITION_CONFIG_DIR = os.path.join(WORK_DIR, "recognition_configs")
LOCAL_RECOGNITION_MODEL_DIR = os.path.join(WORK_DIR, "recognition_models")

# สร้าง directories
for dir_path in [LOCAL_RECOGNITION_DATA_DIR, LOCAL_RECOGNITION_CONFIG_DIR, LOCAL_RECOGNITION_MODEL_DIR]:
    os.makedirs(dir_path, exist_ok=True)

print("📁 S3 Configuration:")
print(f"   Bucket: {S3_BUCKET}")
print(f"   Recognition Data Prefix: {S3_RECOGNITION_DATA_PREFIX}")
print(f"   Recognition Model Prefix: {S3_RECOGNITION_MODEL_PREFIX}")

print("\n📂 Local Paths:")
print(f"   Recognition Data: {LOCAL_RECOGNITION_DATA_DIR}")
print(f"   Recognition Config: {LOCAL_RECOGNITION_CONFIG_DIR}")
print(f"   Recognition Models: {LOCAL_RECOGNITION_MODEL_DIR}")

print("✅ Path configuration completed!")

## 3. Prepare Recognition Annotation Files

สร้างและตรวจสอบไฟล์ annotation สำหรับ Text Recognition ในรูปแบบ: `image_path\ttext_content`

In [ ]:
def create_recognition_annotation_file(annotation_path, num_samples=10):
    """
    สร้างไฟล์ annotation ตัวอย่างสำหรับ Text Recognition
    รูปแบบ: image_path\ttext_content
    """
    sample_annotations = []
    
    # ตัวอย่างข้อมูล Recognition
    sample_texts = [
        "สวัสดีครับ", "PaddleOCR", "Text Recognition", "1234567890",
        "Hello World", "ยินดีต้อนรับ", "深度学习", "Machine Learning",
        "Amazon SageMaker", "การรู้จำตัวอักษร"
    ]
    
    for i in range(num_samples):
        image_path = f"recognition_images/word_{i:03d}.jpg"
        text_content = sample_texts[i % len(sample_texts)]
        
        # สร้างบรรทัดในรูปแบบ Recognition: image_path\ttext_content
        line = f"{image_path}\t{text_content}"
        sample_annotations.append(line)
    
    # เขียนไฟล์
    with open(annotation_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sample_annotations))
    
    print(f"✅ Created recognition annotation file: {annotation_path}")
    return annotation_path

def validate_recognition_annotation_format(annotation_file):
    """
    ตรวจสอบรูปแบบไฟล์ annotation สำหรับ Recognition
    รูปแบบที่ถูกต้อง: image_path\ttext_content
    """
    try:
        with open(annotation_file, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if not line:
                    continue
                
                # แยกด้วย tab
                parts = line.split('\t')
                if len(parts) != 2:
                    print(f"❌ Error at line {line_num}: Expected 2 parts separated by tab, got {len(parts)}")
                    print(f"   Line content: {repr(line)}")
                    return False
                
                image_path, text_content = parts
                
                # ตรวจสอบว่ามีข้อความ
                if not text_content.strip():
                    print(f"❌ Error at line {line_num}: Empty text content")
                    return False
                
                if line_num <= 3:  # แสดงตัวอย่าง 3 บรรทัดแรก
                    print(f"   Line {line_num}: {image_path} -> {text_content}")
        
        print("✅ Recognition annotation format validation passed!")
        return True
    
    except Exception as e:
        print(f"❌ Error validating annotation file: {e}")
        return False

# สร้างไฟล์ annotation ตัวอย่าง
train_annotation_file = os.path.join(LOCAL_RECOGNITION_DATA_DIR, "train_recognition.txt")
val_annotation_file = os.path.join(LOCAL_RECOGNITION_DATA_DIR, "val_recognition.txt")

print("📝 Creating sample recognition annotation files...")
create_recognition_annotation_file(train_annotation_file, 20)
create_recognition_annotation_file(val_annotation_file, 5)

print("\n🔍 Validating annotation files...")
validate_recognition_annotation_format(train_annotation_file)
validate_recognition_annotation_format(val_annotation_file)

## 4. Upload Recognition Data and Annotation to S3

อัปโหลดข้อมูล Recognition (รูปภาพข้อความที่ตัดแล้ว + ไฟล์ annotation) ไปยัง S3

In [ ]:
def upload_recognition_data_to_s3(local_data_path, s3_bucket, s3_prefix):
    """
    อัปโหลดข้อมูล Recognition ไปยัง S3 bucket
    """
    s3_client = boto3.client('s3')
    uploaded_files = []
    
    for root, dirs, files in os.walk(local_data_path):
        for file in files:
            local_file_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_file_path, local_data_path)
            s3_key = f"{s3_prefix}/{relative_path}"
            
            print(f"📤 Uploading: {relative_path} -> s3://{s3_bucket}/{s3_key}")
            s3_client.upload_file(local_file_path, s3_bucket, s3_key)
            uploaded_files.append(s3_key)
    
    print(f"✅ Uploaded {len(uploaded_files)} files to S3")
    return uploaded_files

def create_sample_recognition_images():
    """
    สร้างรูปภาพตัวอย่างสำหรับ Recognition (จำลอง cropped text images)
    """
    import numpy as np
    from PIL import Image, ImageDraw, ImageFont
    
    # สร้างโฟลเดอร์สำหรับรูปภาพ
    images_dir = os.path.join(LOCAL_RECOGNITION_DATA_DIR, "recognition_images")
    os.makedirs(images_dir, exist_ok=True)
    
    # ข้อความตัวอย่าง
    sample_texts = [
        "สวัสดีครับ", "PaddleOCR", "Text Recognition", "1234567890",
        "Hello World", "ยินดีต้อนรับ", "深度学习", "Machine Learning",
        "Amazon SageMaker", "การรู้จำตัวอักษร"
    ]
    
    for i, text in enumerate(sample_texts[:10]):  # สร้าง 10 รูป
        # สร้างรูปภาพ
        img = Image.new('RGB', (200, 50), color='white')
        draw = ImageDraw.Draw(img)
        
        # เขียนข้อความ (ใช้ default font)
        try:
            draw.text((10, 15), text, fill='black')
        except:
            # ถ้าไม่สามารถเขียนข้อความได้ ให้ใช้เลขแทน
            draw.text((10, 15), f"Text{i:03d}", fill='black')
        
        # บันทึกรูปภาพ
        image_path = os.path.join(images_dir, f"word_{i:03d}.jpg")
        img.save(image_path)
        
    print(f"✅ Created {len(sample_texts)} sample recognition images in {images_dir}")

# สร้างรูปภาพตัวอย่าง
print("🖼️ Creating sample recognition images...")
create_sample_recognition_images()

# อัปโหลดข้อมูล Recognition ไปยัง S3
print(f"\n📤 Uploading recognition data to S3 bucket: {S3_BUCKET}")
uploaded_files = upload_recognition_data_to_s3(
    LOCAL_RECOGNITION_DATA_DIR, 
    S3_BUCKET, 
    S3_RECOGNITION_DATA_PREFIX
)

print(f"\n📋 Uploaded files preview (first 10):")
for i, file in enumerate(uploaded_files[:10]):
    print(f"   {i+1}. s3://{S3_BUCKET}/{file}")

if len(uploaded_files) > 10:
    print(f"   ... และอีก {len(uploaded_files) - 10} ไฟล์")

## 5. Download Recognition Data from S3

ดาวน์โหลดข้อมูล Recognition จาก S3 เพื่อใช้ในการเทรน

In [ ]:
def download_recognition_data_from_s3(s3_bucket, s3_prefix, local_data_path):
    """
    ดาวน์โหลดข้อมูล Recognition จาก S3 bucket
    """
    s3_client = boto3.client('s3')
    downloaded_files = []
    
    # สร้าง directory หากไม่มี
    os.makedirs(local_data_path, exist_ok=True)
    
    # ดึงรายการไฟล์จาก S3
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=s3_bucket, Prefix=s3_prefix)
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                s3_key = obj['Key']
                # ลบ prefix เพื่อได้ relative path
                relative_path = s3_key[len(s3_prefix):].lstrip('/')
                local_file_path = os.path.join(local_data_path, relative_path)
                
                # สร้าง directory หากจำเป็น
                os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
                
                print(f"📥 Downloading: s3://{s3_bucket}/{s3_key} -> {relative_path}")
                s3_client.download_file(s3_bucket, s3_key, local_file_path)
                downloaded_files.append(local_file_path)
    
    print(f"✅ Downloaded {len(downloaded_files)} files from S3")
    return downloaded_files

def list_s3_recognition_data(s3_bucket, s3_prefix):
    """
    แสดงรายการไฟล์ Recognition ใน S3
    """
    s3_client = boto3.client('s3')
    
    try:
        response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_prefix)
        
        if 'Contents' in response:
            files = []
            for obj in response['Contents']:
                files.append({
                    'key': obj['Key'],
                    'size': obj['Size'],
                    'modified': obj['LastModified']
                })
            
            print(f"📋 Recognition data in s3://{s3_bucket}/{s3_prefix}:")
            for file_info in sorted(files, key=lambda x: x['key'])[:10]:  # แสดง 10 ไฟล์แรก
                size_mb = file_info['size'] / (1024 * 1024)
                print(f"   📄 {file_info['key']} ({size_mb:.2f} MB)")
                
            if len(files) > 10:
                print(f"   ... และอีก {len(files) - 10} ไฟล์")
                
            return files
        else:
            print("📭 No files found in S3 bucket with the specified prefix.")
            return []
    
    except Exception as e:
        print(f"❌ Error listing S3 files: {e}")
        return []

# แสดงรายการไฟล์ใน S3
print("📋 Listing recognition data in S3:")
s3_files = list_s3_recognition_data(S3_BUCKET, S3_RECOGNITION_DATA_PREFIX)

# สร้าง clean directory สำหรับ download
download_dir = os.path.join(WORK_DIR, "downloaded_recognition_data")
if os.path.exists(download_dir):
    import shutil
    shutil.rmtree(download_dir)

print(f"\n📥 Downloading recognition data to: {download_dir}")
downloaded_files = download_recognition_data_from_s3(
    S3_BUCKET, 
    S3_RECOGNITION_DATA_PREFIX, 
    download_dir
)

# แสดงผลการดาวน์โหลด
print(f"\n📁 Downloaded files structure:")
for root, dirs, files in os.walk(download_dir):
    level = root.replace(download_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}📂 {os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # แสดงเฉพาะ 5 ไฟล์แรกต่อโฟลเดอร์
        print(f"{subindent}📄 {file}")
    if len(files) > 5:
        print(f"{subindent}... และอีก {len(files) - 5} ไฟล์")

## 6. Validate Recognition Annotation Format

ตรวจสอบรูปแบบไฟล์ annotation สำหรับ Recognition ให้ถูกต้อง

In [ ]:
def comprehensive_recognition_validation(annotation_file, image_base_dir=None):
    """
    ตรวจสอบไฟล์ annotation สำหรับ Recognition อย่างครบถ้วน
    """
    print(f"🔍 Validating recognition annotation file: {annotation_file}")
    
    if not os.path.exists(annotation_file):
        print(f"❌ Annotation file not found: {annotation_file}")
        return False
    
    issues = []
    valid_lines = 0
    text_lengths = []
    
    with open(annotation_file, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            
            try:
                # ตรวจสอบรูปแบบ tab-separated
                parts = line.split('\t')
                if len(parts) != 2:
                    issues.append(f"Line {line_num}: Expected 2 parts (image_path\\ttext), got {len(parts)}")
                    continue
                
                image_path, text_content = parts
                
                # ตรวจสอบ image path
                if not image_path.strip():
                    issues.append(f"Line {line_num}: Empty image path")
                    continue
                
                # ตรวจสอบ text content
                if not text_content.strip():
                    issues.append(f"Line {line_num}: Empty text content")
                    continue
                
                # ตรวจสอบการมีอยู่ของไฟล์รูปภาพ (ถ้าระบุ base directory)
                if image_base_dir:
                    full_image_path = os.path.join(image_base_dir, image_path)
                    if not os.path.exists(full_image_path):
                        issues.append(f"Line {line_num}: Image file not found: {image_path}")
                
                # เก็บสถิติ
                text_lengths.append(len(text_content))
                valid_lines += 1
                
                # แสดงตัวอย่าง
                if valid_lines <= 5:
                    print(f"   ✓ Line {line_num}: {image_path} -> '{text_content}'")
                
            except Exception as e:
                issues.append(f"Line {line_num}: Error processing line - {str(e)}")
    
    # สรุปผลการตรวจสอบ
    print(f"\n📊 Validation Summary:")
    print(f"   Valid lines: {valid_lines}")
    print(f"   Issues found: {len(issues)}")
    
    if text_lengths:
        import numpy as np
        print(f"   Text length stats:")
        print(f"     Min: {min(text_lengths)} characters")
        print(f"     Max: {max(text_lengths)} characters")
        print(f"     Average: {np.mean(text_lengths):.1f} characters")
    
    # แสดงปัญหาที่พบ
    if issues:
        print(f"\n❌ Issues found:")
        for issue in issues[:10]:  # แสดงเฉพาะ 10 ปัญหาแรก
            print(f"   • {issue}")
        if len(issues) > 10:
            print(f"   ... และอีก {len(issues) - 10} ปัญหา")
        return False
    else:
        print("✅ All validation checks passed!")
        return True

def convert_detection_to_recognition_format(detection_annotation_file, output_recognition_file):
    """
    แปลงไฟล์ annotation จาก Detection format เป็น Recognition format
    จาก: image_path\t[{"transcription": "text", "points": [...]}]
    เป็น: image_path\ttext
    """
    print(f"🔄 Converting detection annotation to recognition format...")
    
    recognition_lines = []
    
    with open(detection_annotation_file, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            
            try:
                parts = line.split('\t')
                if len(parts) != 2:
                    continue
                
                image_path, annotation_json = parts
                annotations = json.loads(annotation_json)
                
                # รวมข้อความทั้งหมดในรูปภาพ
                texts = []
                for ann in annotations:
                    if 'transcription' in ann:
                        texts.append(ann['transcription'])
                
                if texts:
                    # รวมข้อความด้วยช่องว่าง
                    combined_text = ' '.join(texts)
                    recognition_line = f"{image_path}\t{combined_text}"
                    recognition_lines.append(recognition_line)
                
            except json.JSONDecodeError:
                print(f"Warning: Invalid JSON at line {line_num}")
                continue
            except Exception as e:
                print(f"Warning: Error processing line {line_num}: {e}")
                continue
    
    # เขียนไฟล์ผลลัพธ์
    with open(output_recognition_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(recognition_lines))
    
    print(f"✅ Converted {len(recognition_lines)} lines to recognition format")
    print(f"   Output file: {output_recognition_file}")
    
    return output_recognition_file

# ค้นหาไฟล์ annotation ที่ดาวน์โหลดมา
downloaded_annotation_files = []
for root, dirs, files in os.walk(download_dir):
    for file in files:
        if file.endswith('.txt'):
            downloaded_annotation_files.append(os.path.join(root, file))

print(f"📋 Found annotation files to validate:")
for i, file_path in enumerate(downloaded_annotation_files):
    print(f"   {i+1}. {os.path.relpath(file_path, download_dir)}")

# ตรวจสอบไฟล์ annotation แต่ละไฟล์
print(f"\n🔍 Validating annotation files...")
for annotation_file in downloaded_annotation_files:
    print(f"\n{'='*60}")
    
    # หาโฟลเดอร์รูปภาพที่เกี่ยวข้อง
    annotation_dir = os.path.dirname(annotation_file)
    possible_image_dirs = [
        os.path.join(annotation_dir, 'images'),
        os.path.join(annotation_dir, 'recognition_images'),
        annotation_dir
    ]
    
    image_base_dir = None
    for img_dir in possible_image_dirs:
        if os.path.exists(img_dir) and any(f.lower().endswith(('.jpg', '.jpeg', '.png')) 
                                          for f in os.listdir(img_dir)):
            image_base_dir = img_dir
            break
    
    # ทำการตรวจสอบ
    is_valid = comprehensive_recognition_validation(annotation_file, image_base_dir)

## 7. Prepare Recognition Configuration File (.yml)

สร้างและปรับแต่งไฟล์ configuration สำหรับการเทรน Text Recognition

In [ ]:
def create_recognition_training_config(base_config_path, output_config_path, 
                                     train_annotation_file, eval_annotation_file,
                                     save_model_dir, architecture_type='CRNN'):
    """
    สร้างไฟล์ configuration สำหรับการเทรน Recognition
    
    Args:
        base_config_path: Path ไปยัง base config file
        output_config_path: Path สำหรับบันทึก config ที่แก้ไขแล้ว
        train_annotation_file: Path ไปยังไฟล์ annotation สำหรับ training
        eval_annotation_file: Path ไปยังไฟล์ annotation สำหรับ evaluation
        save_model_dir: Directory สำหรับบันทึก model checkpoints
        architecture_type: ประเภทของ architecture ('CRNN', 'SVTR', 'PP-OCRv4')
    """
    
    # โหลด base configuration
    with open(base_config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    print(f"📄 Loaded base config from: {base_config_path}")
    print(f"   Architecture type: {architecture_type}")
    
    # อัปเดต Global settings
    config['Global']['save_model_dir'] = save_model_dir
    config['Global']['checkpoints'] = None  # Start from scratch
    config['Global']['use_gpu'] = True
    config['Global']['epoch_num'] = 100  # ลดจำนวน epoch สำหรับ demo
    config['Global']['log_smooth_window'] = 20
    config['Global']['print_batch_step'] = 10
    config['Global']['save_epoch_step'] = 20
    config['Global']['eval_batch_step'] = [0, 500]
    config['Global']['cal_metric_during_train'] = True
    
    # ตั้งค่า character dictionary (ปรับตามภาษาที่ใช้)
    config['Global']['character_dict_path'] = 'ppocr/utils/ppocr_keys_v1.txt'
    config['Global']['character_type'] = 'ch'  # ch = Chinese/Thai, en = English
    config['Global']['max_text_length'] = 25
    
    # อัปเดต Train dataset
    config['Train']['dataset']['name'] = 'SimpleDataSet'
    config['Train']['dataset']['data_dir'] = os.path.dirname(train_annotation_file)
    config['Train']['dataset']['label_file_list'] = [train_annotation_file]
    
    # อัปเดต Eval dataset
    config['Eval']['dataset']['name'] = 'SimpleDataSet'
    config['Eval']['dataset']['data_dir'] = os.path.dirname(eval_annotation_file)
    config['Eval']['dataset']['label_file_list'] = [eval_annotation_file]
    
    # ปรับแต่ง Optimizer
    if 'Optimizer' in config:
        config['Optimizer']['lr']['learning_rate'] = 0.001
        if 'regularizer' in config['Optimizer']:
            config['Optimizer']['regularizer']['factor'] = 5.0e-05
    
    # ปรับแต่ง batch size ตาม GPU memory
    if 'Train' in config and 'loader' in config['Train']:
        config['Train']['loader']['batch_size_per_card'] = 32  # ลดถ้า GPU memory ไม่พอ
        config['Train']['loader']['num_workers'] = 4
    
    if 'Eval' in config and 'loader' in config['Eval']:
        config['Eval']['loader']['batch_size_per_card'] = 32
        config['Eval']['loader']['num_workers'] = 2
    
    # บันทึกไฟล์ config ที่แก้ไขแล้ว
    os.makedirs(os.path.dirname(output_config_path), exist_ok=True)
    with open(output_config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print(f"✅ Created recognition training config: {output_config_path}")
    
    # แสดงการตั้งค่าสำคัญ
    print(f\"\\n📋 Key Configuration Settings:\")
    print(f\"   Model save dir: {config['Global']['save_model_dir']}\")
    print(f\"   Epochs: {config['Global']['epoch_num']}\")
    print(f\"   Learning rate: {config.get('Optimizer', {}).get('lr', {}).get('learning_rate', 'Not set')}\")
    print(f\"   Train data: {train_annotation_file}\")
    print(f\"   Eval data: {eval_annotation_file}\")
    print(f\"   Character type: {config['Global']['character_type']}\")
    
    return config

def list_available_recognition_configs():
    \"\"\"
    แสดงรายการ config files ที่มีสำหรับ Recognition
    \"\"\"
    rec_config_dir = os.path.join(PADDLEOCR_DIR, \"configs\", \"rec\")
    
    if not os.path.exists(rec_config_dir):
        print(f\"❌ Recognition config directory not found: {rec_config_dir}\")
        return []
    
    config_files = []
    for root, dirs, files in os.walk(rec_config_dir):
        for file in files:
            if file.endswith('.yml'):
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, rec_config_dir)
                config_files.append({
                    'name': relative_path,
                    'full_path': full_path,
                    'architecture': 'Unknown'
                })
    
    # พยายามระบุ architecture type
    for config in config_files:
        name_lower = config['name'].lower()
        if 'crnn' in name_lower or 'bilstm' in name_lower:
            config['architecture'] = 'CRNN'
        elif 'svtr' in name_lower:
            config['architecture'] = 'SVTR'
        elif 'ocrv4' in name_lower or 'pp-ocrv4' in name_lower:
            config['architecture'] = 'PP-OCRv4'
        elif 'rare' in name_lower:
            config['architecture'] = 'RareText'
        elif 'vitstr' in name_lower:
            config['architecture'] = 'ViTSTR'
    
    return config_files

# แสดงรายการ config ที่มี
print(\"🔍 Available Recognition Configuration Files:\")
available_configs = list_available_recognition_configs()

if available_configs:
    print(f\"\\n📋 Found {len(available_configs)} recognition configs:\")
    for i, config in enumerate(available_configs[:10]):  # แสดง 10 อันแรก
        print(f\"   {i+1}. {config['name']} ({config['architecture']})\")
    
    if len(available_configs) > 10:
        print(f\"   ... และอีก {len(available_configs) - 10} configs\")
    
    # เลือก config ที่แนะนำ
    recommended_configs = [
        ('rec_mv3_none_bilstm_ctc.yml', 'CRNN - Basic, fast training'),
        ('rec_svtr_base.yml', 'SVTR - Better accuracy'),
        ('PP-OCRv4/en_PP-OCRv4_rec.yml', 'PP-OCRv4 - Latest model')
    ]
    
    print(f\"\\n🌟 Recommended configs:\")
    for config_name, description in recommended_configs:
        found = next((c for c in available_configs if config_name in c['name']), None)
        if found:
            print(f\"   ✓ {config_name} - {description}\")
            print(f\"     Path: {found['full_path']}\")
        else:
            print(f\"   ❌ {config_name} - Not found\")
            
else:
    print(\"❌ No recognition config files found!\")

In [ ]:
# เลือกและสร้าง recognition config file
# ใช้ CRNN config เป็นค่าเริ่มต้น (เร็วและเสถียร)
base_config_name = "rec_mv3_none_bilstm_ctc.yml"
base_config_path = os.path.join(PADDLEOCR_DIR, "configs", "rec", base_config_name)

# ตรวจสอบว่ามี config file หรือไม่
if not os.path.exists(base_config_path):
    print(f"❌ Base config not found: {base_config_path}")
    # ลองหา config อื่น
    alternative_configs = [
        "configs/rec/rec_r34_vd_none_bilstm_ctc.yml",
        "configs/rec/rec_mv3_none_none_ctc.yml"
    ]
    
    for alt_config in alternative_configs:
        alt_path = os.path.join(PADDLEOCR_DIR, alt_config)
        if os.path.exists(alt_path):
            base_config_path = alt_path
            print(f"✅ Using alternative config: {alt_path}")
            break
    else:
        print("❌ No suitable recognition config found!")

if os.path.exists(base_config_path):
    # ตั้งค่า paths สำหรับ training
    train_annotation_path = os.path.join(download_dir, "train_recognition.txt")
    eval_annotation_path = os.path.join(download_dir, "val_recognition.txt")
    
    # ตรวจสอบว่าไฟล์ annotation มีอยู่
    if not os.path.exists(train_annotation_path):
        print(f"❌ Train annotation file not found: {train_annotation_path}")
        # ใช้ไฟล์ที่สร้างไว้ก่อนหน้า
        train_annotation_path = os.path.join(LOCAL_RECOGNITION_DATA_DIR, "train_recognition.txt")
        eval_annotation_path = os.path.join(LOCAL_RECOGNITION_DATA_DIR, "val_recognition.txt")
    
    # สร้างไฟล์ config สำหรับ training
    output_config_path = os.path.join(LOCAL_RECOGNITION_CONFIG_DIR, "recognition_training_config.yml")
    
    print(f"🔧 Creating recognition training configuration...")
    print(f"   Base config: {base_config_path}")
    print(f"   Output config: {output_config_path}")
    print(f"   Train data: {train_annotation_path}")
    print(f"   Eval data: {eval_annotation_path}")
    print(f"   Model save dir: {LOCAL_RECOGNITION_MODEL_DIR}")
    
    # สร้าง config
    config = create_recognition_training_config(
        base_config_path=base_config_path,
        output_config_path=output_config_path,
        train_annotation_file=train_annotation_path,
        eval_annotation_file=eval_annotation_path,
        save_model_dir=LOCAL_RECOGNITION_MODEL_DIR,
        architecture_type='CRNN'
    )
    
    # ตรวจสอบความถูกต้องของ config ที่สร้างแล้ว
    print(f"\n🔍 Validating created config file...")
    try:
        with open(output_config_path, 'r', encoding='utf-8') as f:
            test_config = yaml.safe_load(f)
        
        required_sections = ['Global', 'Architecture', 'Loss', 'Optimizer', 'Train', 'Eval']
        missing_sections = [section for section in required_sections if section not in test_config]
        
        if missing_sections:
            print(f"❌ Missing config sections: {missing_sections}")
        else:
            print("✅ Config validation passed!")
            
            # แสดง architecture details
            if 'Architecture' in test_config:
                arch = test_config['Architecture']
                print(f"\\n🏗️ Architecture Details:")
                print(f"   Model type: {arch.get('model_type', 'Not specified')}")
                print(f"   Algorithm: {arch.get('algorithm', 'Not specified')}")
                if 'Backbone' in arch:
                    print(f"   Backbone: {arch['Backbone'].get('name', 'Not specified')}")
                if 'Head' in arch:
                    print(f"   Head: {arch['Head'].get('name', 'Not specified')}")
    
    except Exception as e:
        print(f"❌ Error validating config: {e}")

else:
    print("❌ Cannot proceed without a valid base config file!")
    
print(f"\\n✅ Recognition configuration setup completed!")

## 8. Start Text Recognition Training with tools/train_rec.py

เริ่มการเทรน Text Recognition โดยใช้ script `tools/train_rec.py` เท่านั้น

In [ ]:
def start_recognition_training(config_path, resume_from=None):
    """
    เริ่มการเทรน Text Recognition โดยใช้ tools/train_rec.py
    
    Args:
        config_path: Path ไปยังไฟล์ configuration
        resume_from: Path ไปยัง checkpoint สำหรับ resume (optional)
    """
    
    # เปลี่ยนไปยัง PaddleOCR directory
    os.chdir(PADDLEOCR_DIR)
    
    # เตรียมคำสั่งสำหรับการเทรน Recognition
    train_script = os.path.join(PADDLEOCR_DIR, "tools", "train.py")  # ใช้ tools/train.py ธรรมดา
    
    # ตรวจสอบว่ามี train_rec.py หรือไม่
    train_rec_script = os.path.join(PADDLEOCR_DIR, "tools", "train_rec.py")
    if os.path.exists(train_rec_script):
        train_script = train_rec_script
        print(f"🎯 Using specialized recognition training script: train_rec.py")
    else:
        print(f"📝 Using general training script: train.py")
    
    cmd = [sys.executable, train_script, "-c", config_path]
    
    if resume_from:
        cmd.extend(["-o", f"Global.checkpoints={resume_from}"])
    
    print(f"🚀 Starting Recognition Training...")
    print(f"   Command: {' '.join(cmd)}")
    print(f"   Config: {config_path}")
    print(f"   Working directory: {os.getcwd()}")
    
    # สร้าง log file
    log_file = os.path.join(LOCAL_RECOGNITION_MODEL_DIR, "training.log")
    
    try:
        # รันการเทรนแบบ non-blocking เพื่อให้ดู output ได้
        print(f"📊 Training output will be saved to: {log_file}")
        print(f"🔄 Training started... (This may take a while)")
        
        with open(log_file, 'w', encoding='utf-8') as log_f:
            process = subprocess.Popen(
                cmd, 
                stdout=subprocess.PIPE, 
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1
            )
            
            # แสดง output แบบ real-time
            line_count = 0
            for line in iter(process.stdout.readline, ''):
                line_count += 1
                print(line.strip())
                log_f.write(line)
                log_f.flush()
                
                # หยุดแสดง output หลังจาก 100 บรรทัดเพื่อไม่ให้ notebook ยาวเกินไป
                if line_count > 100:
                    print("\\n... (Training continues, check log file for full output)")
                    break
            
            # รอให้ process เสร็จ (สำหรับ demo ให้รอสักพัก)
            process.wait()
            
            if process.returncode == 0:
                print("✅ Training completed successfully!")
                return True
            else:
                print(f"❌ Training failed with return code: {process.returncode}")
                return False
                
    except KeyboardInterrupt:
        print("\\n⏹️ Training interrupted by user")
        return False
    except Exception as e:
        print(f"❌ Training failed with error: {e}")
        return False

def monitor_recognition_training_progress(model_save_dir):
    """
    ติดตามความคืบหน้าของการเทรน Recognition
    """
    print(f"📊 Monitoring training progress in: {model_save_dir}")
    
    if not os.path.exists(model_save_dir):
        print("📁 Model save directory doesn't exist yet.")
        return
    
    # หา checkpoint files
    checkpoints = []
    for file in os.listdir(model_save_dir):
        if file.endswith('.pdparams'):
            checkpoint_path = os.path.join(model_save_dir, file)
            mtime = os.path.getmtime(checkpoint_path)
            size = os.path.getsize(checkpoint_path) / (1024 * 1024)  # MB
            checkpoints.append({
                'name': file,
                'path': checkpoint_path,
                'modified': datetime.fromtimestamp(mtime),
                'size_mb': size
            })
    
    if checkpoints:
        checkpoints.sort(key=lambda x: x['modified'], reverse=True)
        print(f"\\n🎯 Found {len(checkpoints)} Recognition model checkpoints:")
        
        for i, checkpoint in enumerate(checkpoints[:5]):  # แสดง 5 ไฟล์ล่าสุด
            print(f"   {i+1}. {checkpoint['name']}")
            print(f"      Size: {checkpoint['size_mb']:.1f} MB")
            print(f"      Modified: {checkpoint['modified']}")
        
        if len(checkpoints) > 5:
            print(f"   ... และอีก {len(checkpoints) - 5} checkpoints")
            
        # แสดงข้อมูล latest checkpoint
        latest = checkpoints[0]
        print(f"\\n📈 Latest checkpoint: {latest['name']}")
        print(f"   Created: {latest['modified']}")
        print(f"   Size: {latest['size_mb']:.1f} MB")
        
    else:
        print("📭 No checkpoints found yet.")
    
    # ตรวจสอบ log file
    log_file = os.path.join(model_save_dir, "training.log")
    if os.path.exists(log_file):
        print(f"\\n📝 Training log preview (last 10 lines):")
        try:
            with open(log_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                for line in lines[-10:]:
                    print(f"   {line.strip()}")
        except:
            print("   (Unable to read log file)")

# ตรวจสอบความพร้อมก่อนเริ่มเทรน
print("🔍 Pre-training checklist:")
checklist_items = [
    ("PaddleOCR repository", os.path.exists(PADDLEOCR_DIR)),
    ("Training script", os.path.exists(os.path.join(PADDLEOCR_DIR, "tools", "train.py"))),
    ("Configuration file", os.path.exists(output_config_path)),
    ("Train annotation", os.path.exists(train_annotation_path)),
    ("Eval annotation", os.path.exists(eval_annotation_path)),
    ("Model save directory", os.path.exists(LOCAL_RECOGNITION_MODEL_DIR)),
    ("GPU available", paddle.is_compiled_with_cuda())
]

all_ready = True
for item_name, is_ready in checklist_items:
    status = "✅" if is_ready else "❌"
    print(f"   {status} {item_name}")
    if not is_ready:
        all_ready = False

if all_ready:
    print("\\n🚀 All checks passed! Ready to start training.")
    
    # เริ่มการเทรน (แสดงตัวอย่างคำสั่ง)
    print(f"\\n📋 Training command that will be executed:")
    print(f"   cd {PADDLEOCR_DIR}")
    print(f"   python tools/train.py -c {output_config_path}")
    
    # ถาม user ว่าต้องการเริ่มเทรนหรือไม่
    print(f"\\n⚠️  Note: Training will take time and GPU resources.")
    print(f"   Uncomment the following lines to start actual training:")
    print(f"   # training_success = start_recognition_training(output_config_path)")
    print(f"   # monitor_recognition_training_progress(LOCAL_RECOGNITION_MODEL_DIR)")
    
else:
    print("\\n❌ Some requirements are not met. Please fix issues before training.")

### 🚀 Ready-to-Execute Training Commands

เลือกวิธีการเทรนที่เหมาะสม

In [ ]:
# ============================================================================
# 🎯 TRAINING EXECUTION OPTIONS
# ============================================================================

print("🚀 RECOGNITION TRAINING EXECUTION OPTIONS")
print("="*55)

if all_ready:
    print("\n✅ All prerequisites are ready!")
    
    # Option 1: Quick demo training (short)
    print(f"\n🚀 OPTION 1: Quick Demo Training (2-3 epochs)")
    print(f"   Perfect for testing and verification")
    print(f"   Uncomment to run:")
    print(f"   # quick_config = create_quick_demo_config()")
    print(f"   # training_success = start_recognition_training(quick_config)")
    
    # Option 2: Full production training
    print(f"\n🏭 OPTION 2: Full Production Training")
    print(f"   Complete training with many epochs")
    print(f"   Uncomment to run:")
    print(f"   # training_success = start_recognition_training(output_config_path)")
    print(f"   # monitor_recognition_training_progress(LOCAL_RECOGNITION_MODEL_DIR)")
    
    # Option 3: Manual command line
    print(f"\n💻 OPTION 3: Manual Command Line Execution")
    print(f"   Run in terminal for full control:")
    print(f"   cd {PADDLEOCR_DIR}")
    print(f"   python tools/train.py -c {output_config_path}")
    
    # Option 4: Background training
    print(f"\n🔄 OPTION 4: Background Training with Monitoring")
    print(f"   Training runs in background, periodic monitoring")
    print(f"   # start_background_recognition_training()")
    
    print(f"\n⚡ CHOOSE YOUR TRAINING METHOD:")
    print(f"   1. For testing: Use Option 1 (Quick Demo)")
    print(f"   2. For production: Use Option 2 (Full Training)")
    print(f"   3. For control: Use Option 3 (Manual)")
    print(f"   4. For long runs: Use Option 4 (Background)")

else:
    print("\n❌ Prerequisites not met. Please complete setup first.")
    print("    Check previous sections for missing requirements.")

print(f"\n" + "="*55)

# Quick demo config function
def create_quick_demo_config():
    """
    สร้าง config สำหรับ demo training (epochs น้อย)
    """
    # Copy config ปกติแล้วแก้ไข epochs
    demo_config_path = output_config_path.replace('.yml', '_demo.yml')
    
    with open(output_config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    # ลด epochs เพื่อ demo
    config['Global']['epoch_num'] = 3
    config['Global']['eval_batch_step'] = [0, 5, 10]
    config['Global']['save_epoch_step'] = 1
    
    # เขียนไฟล์ config ใหม่
    with open(demo_config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, default_flow_style=False, allow_unicode=True)
    
    print(f"✅ Created demo config: {demo_config_path}")
    print(f"   • Epochs: 3 (vs original {config.get('Global', {}).get('epoch_num', 'unknown')})")
    print(f"   • Quick evaluation and saving")
    
    return demo_config_path

def start_background_recognition_training():
    """
    เริ่มการเทรนแบบ background พร้อม monitoring
    """
    print(f"🔄 Starting background Recognition training...")
    
    # เริ่มการเทรนแบบ background
    log_file = os.path.join(LOCAL_RECOGNITION_MODEL_DIR, "training.log")
    
    train_cmd = [
        sys.executable, 
        os.path.join(PADDLEOCR_DIR, "tools", "train.py"), 
        "-c", output_config_path
    ]
    
    try:
        # Start background process
        with open(log_file, 'w') as log_f:
            process = subprocess.Popen(
                train_cmd,
                cwd=PADDLEOCR_DIR,
                stdout=log_f,
                stderr=subprocess.STDOUT
            )
        
        print(f"✅ Training started in background (PID: {process.pid})")
        print(f"📊 Monitor log: tail -f {log_file}")
        print(f"⏹️  Stop training: kill {process.pid}")
        
        return process
        
    except Exception as e:
        print(f"❌ Failed to start background training: {e}")
        return None

# Ready-to-execute command examples
print(f"\n📋 READY-TO-EXECUTE EXAMPLES:")
print(f"\n# Quick 3-epoch demo:")
print(f"# demo_config = create_quick_demo_config()")
print(f"# start_recognition_training(demo_config)")
print(f"\n# Full production training:")
print(f"# start_recognition_training(output_config_path)")
print(f"\n# Background training:")
print(f"# bg_process = start_background_recognition_training()")
print(f"\n# Monitor progress:")
print(f"# monitor_recognition_training_progress(LOCAL_RECOGNITION_MODEL_DIR)")

print(f"\n💡 TIP: Start with quick demo to verify everything works!")
print(f"    Then proceed to full training for production models.")

## 9. Sync Recognition Model Checkpoints to S3

อัปโหลด Recognition model checkpoints (ไฟล์ .pdparams และ .pdopt) ไปยัง S3

In [ ]:
def sync_recognition_checkpoints_to_s3(local_model_dir, s3_bucket, s3_model_prefix):
    """
    Sync Recognition model checkpoints ไปยัง S3
    เฉพาะไฟล์ .pdparams และ .pdopt เท่านั้น
    """
    s3_client = boto3.client('s3')
    
    if not os.path.exists(local_model_dir):
        print(f"❌ Model directory doesn't exist: {local_model_dir}")
        return []
    
    uploaded_files = []
    
    # หาไฟล์ model ที่ต้องอัปโหลด
    for root, dirs, files in os.walk(local_model_dir):
        for file in files:
            # อัปโหลดเฉพาะไฟล์ Recognition model
            if file.endswith(('.pdparams', '.pdopt', '.states', '.yml', '.log')):
                local_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(local_file_path, local_model_dir)
                s3_key = f"{s3_model_prefix}/{relative_path}"
                
                try:
                    print(f"📤 Uploading: {relative_path} -> s3://{s3_bucket}/{s3_key}")
                    s3_client.upload_file(local_file_path, s3_bucket, s3_key)
                    
                    # เพิ่มข้อมูลไฟล์
                    file_size = os.path.getsize(local_file_path) / (1024 * 1024)  # MB
                    uploaded_files.append({
                        'local_path': local_file_path,
                        's3_key': s3_key,
                        'size_mb': file_size,
                        'type': 'recognition_model' if file.endswith('.pdparams') else 'other'
                    })
                    
                except Exception as e:
                    print(f"❌ Failed to upload {file}: {e}")
    
    print(f"\\n✅ Uploaded {len(uploaded_files)} files to S3")
    
    # แยกประเภทไฟล์ที่อัปโหลด
    model_files = [f for f in uploaded_files if f['type'] == 'recognition_model']
    other_files = [f for f in uploaded_files if f['type'] == 'other']
    
    if model_files:
        print(f"\\n🎯 Recognition Model Files ({len(model_files)}):")
        for file_info in model_files:
            print(f"   📊 {os.path.basename(file_info['s3_key'])} ({file_info['size_mb']:.1f} MB)")
    
    if other_files:
        print(f"\\n📋 Other Files ({len(other_files)}):")
        for file_info in other_files[:5]:  # แสดง 5 ไฟล์แรก
            print(f"   📄 {os.path.basename(file_info['s3_key'])} ({file_info['size_mb']:.1f} MB)")
        if len(other_files) > 5:
            print(f"   ... และอีก {len(other_files) - 5} ไฟล์")
    
    return uploaded_files

def download_recognition_checkpoints_from_s3(s3_bucket, s3_model_prefix, local_model_dir):
    """
    ดาวน์โหลด Recognition model checkpoints จาก S3
    """
    s3_client = boto3.client('s3')
    downloaded_files = []
    
    # สร้าง directory หากไม่มี
    os.makedirs(local_model_dir, exist_ok=True)
    
    try:
        # ดึงรายการไฟล์จาก S3
        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=s3_bucket, Prefix=s3_model_prefix)
        
        for page in pages:
            if 'Contents' in page:
                for obj in page['Contents']:
                    s3_key = obj['Key']
                    
                    # ดาวน์โหลดเฉพาะไฟล์ที่เกี่ยวข้องกับ Recognition model
                    if s3_key.endswith(('.pdparams', '.pdopt', '.states', '.yml', '.log')):
                        relative_path = s3_key[len(s3_model_prefix):].lstrip('/')
                        local_file_path = os.path.join(local_model_dir, relative_path)
                        
                        # สร้าง directory หากจำเป็น
                        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
                        
                        print(f"📥 Downloading: s3://{s3_bucket}/{s3_key} -> {relative_path}")
                        s3_client.download_file(s3_bucket, s3_key, local_file_path)
                        downloaded_files.append(local_file_path)
        
        print(f"✅ Downloaded {len(downloaded_files)} files from S3")
        return downloaded_files
        
    except Exception as e:
        print(f"❌ Error downloading checkpoints from S3: {e}")
        return []

def list_s3_recognition_checkpoints(s3_bucket, s3_model_prefix):
    """
    แสดงรายการ Recognition checkpoints ใน S3
    """
    s3_client = boto3.client('s3')
    
    try:
        response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_model_prefix)
        
        if 'Contents' in response:
            checkpoints = []
            for obj in response['Contents']:
                if obj['Key'].endswith(('.pdparams', '.pdopt', '.states')):
                    checkpoints.append({
                        'key': obj['Key'],
                        'size': obj['Size'],
                        'modified': obj['LastModified']
                    })
            
            if checkpoints:
                print(f"📋 Recognition checkpoints in s3://{s3_bucket}/{s3_model_prefix}:")
                for checkpoint in sorted(checkpoints, key=lambda x: x['modified'], reverse=True):
                    size_mb = checkpoint['size'] / (1024 * 1024)
                    print(f"   🎯 {os.path.basename(checkpoint['key'])} ({size_mb:.1f} MB, {checkpoint['modified']})")
                return checkpoints
            else:
                print("📭 No recognition checkpoints found in S3.")
                return []
        else:
            print("📭 No objects found in S3 bucket with the specified prefix.")
            return []
    
    except Exception as e:
        print(f"❌ Error listing S3 checkpoints: {e}")
        return []

# ตรวจสอบ checkpoint files ใน local directory
print("🔍 Checking for Recognition model files in local directory...")
monitor_recognition_training_progress(LOCAL_RECOGNITION_MODEL_DIR)

# อัปโหลด checkpoints ไปยัง S3 (ถ้ามี)
print(f"\\n📤 Syncing Recognition checkpoints to S3...")
uploaded_files = sync_recognition_checkpoints_to_s3(
    LOCAL_RECOGNITION_MODEL_DIR, 
    S3_BUCKET, 
    S3_RECOGNITION_MODEL_PREFIX
)

# แสดงรายการ checkpoints ใน S3
print(f"\\n📋 Listing Recognition checkpoints in S3...")
s3_checkpoints = list_s3_recognition_checkpoints(S3_BUCKET, S3_RECOGNITION_MODEL_PREFIX)

# สร้าง summary
print(f"\\n📊 Recognition Model Sync Summary:")
print(f"   Local model directory: {LOCAL_RECOGNITION_MODEL_DIR}")
print(f"   S3 bucket: s3://{S3_BUCKET}/{S3_RECOGNITION_MODEL_PREFIX}")
print(f"   Files uploaded: {len(uploaded_files)}")
print(f"   Recognition checkpoints in S3: {len(s3_checkpoints)}")

# แสดงคำแนะนำ
print(f"\\n💡 Tips:")
print(f"   - Recognition model files (.pdparams) contain the trained weights")
print(f"   - Optimizer files (.pdopt) contain optimizer state for resuming training")
print(f"   - Use the latest .pdparams file for inference")
print(f"   - Download checkpoints for deployment or further training")

## 10. Monitor Training Progress and Model Output

ติดตามความคืบหน้าการเทรนและตรวจสอบผลลัพธ์ Recognition model

In [ ]:
def monitor_gpu_usage():
    """
    ติดตาม GPU usage ระหว่างการเทรน
    """
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=utilization.gpu,memory.used,memory.total,temperature.gpu', 
                               '--format=csv,noheader,nounits'], capture_output=True, text=True)
        
        if result.returncode == 0:
            lines = result.stdout.strip().split('\\n')
            print(f"🎮 GPU Status:")
            for i, line in enumerate(lines):
                gpu_util, mem_used, mem_total, temp = line.split(', ')
                mem_usage_pct = (int(mem_used) / int(mem_total)) * 100
                print(f"   GPU {i}: {gpu_util}% utilization, {mem_usage_pct:.1f}% memory ({mem_used}/{mem_total} MB), {temp}°C")
        else:
            print("❌ Could not get GPU status")
            
    except Exception as e:
        print(f"❌ Error monitoring GPU: {e}")

def analyze_training_logs(log_file_path):
    """
    วิเคราะห์ log files จากการเทรน Recognition
    """
    if not os.path.exists(log_file_path):
        print(f"📭 Log file not found: {log_file_path}")
        return
    
    print(f"📊 Analyzing training logs: {log_file_path}")
    
    try:
        with open(log_file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # หาข้อมูลสำคัญจาก logs
        loss_values = []
        accuracy_values = []
        lr_values = []
        
        for line in lines:
            line = line.strip()
            
            # หา loss values
            if 'loss:' in line.lower():
                try:
                    loss_part = line.split('loss:')[1].split()[0]
                    loss_val = float(loss_part.replace(',', ''))
                    loss_values.append(loss_val)
                except:
                    pass
            
            # หา accuracy values
            if 'acc:' in line.lower():
                try:
                    acc_part = line.split('acc:')[1].split()[0]
                    acc_val = float(acc_part.replace(',', ''))
                    accuracy_values.append(acc_val)
                except:
                    pass
            
            # หา learning rate
            if 'lr:' in line.lower():
                try:
                    lr_part = line.split('lr:')[1].split()[0]
                    lr_val = float(lr_part.replace(',', ''))
                    lr_values.append(lr_val)
                except:
                    pass
        
        # แสดงสถิติ
        print(f"\\n📈 Training Statistics:")
        
        if loss_values:
            print(f"   Loss: {len(loss_values)} values")
            print(f"     Initial: {loss_values[0]:.4f}")
            print(f"     Final: {loss_values[-1]:.4f}")
            print(f"     Best: {min(loss_values):.4f}")
        
        if accuracy_values:
            print(f"   Accuracy: {len(accuracy_values)} values")
            print(f"     Initial: {accuracy_values[0]:.4f}")
            print(f"     Final: {accuracy_values[-1]:.4f}")
            print(f"     Best: {max(accuracy_values):.4f}")
        
        if lr_values:
            print(f"   Learning Rate: {len(lr_values)} values")
            print(f"     Initial: {lr_values[0]:.6f}")
            print(f"     Final: {lr_values[-1]:.6f}")
        
        # แสดงบรรทัดสุดท้าย 10 บรรทัด
        print(f"\\n📝 Recent log entries:")
        for line in lines[-10:]:
            print(f"   {line.strip()}")
            
    except Exception as e:
        print(f"❌ Error analyzing logs: {e}")

def verify_recognition_model_output():
    """
    ตรวจสอบว่าได้ Recognition model ที่ถูกต้องหรือไม่
    """
    print(f"🔍 Verifying Recognition model output...")
    
    model_files = []
    for root, dirs, files in os.walk(LOCAL_RECOGNITION_MODEL_DIR):
        for file in files:
            if file.endswith('.pdparams'):
                model_files.append({
                    'name': file,
                    'path': os.path.join(root, file),
                    'size': os.path.getsize(os.path.join(root, file)) / (1024 * 1024)
                })
    
    if model_files:
        print(f"✅ Found {len(model_files)} Recognition model files:")
        for model in sorted(model_files, key=lambda x: x['name']):
            print(f"   🎯 {model['name']} ({model['size']:.1f} MB)")
            
            # ตรวจสอบว่าเป็น Recognition model จริงหรือไม่
            if any(keyword in model['name'].lower() for keyword in ['rec', 'recognition', 'crnn', 'svtr']):
                print(f"      ✓ Confirmed as Recognition model")
            else:
                print(f"      ⚠️  Model type unclear from filename")
        
        # แนะนำ model ที่ดีที่สุด
        latest_model = max(model_files, key=lambda x: os.path.getmtime(x['path']))
        print(f"\\n🌟 Recommended model for inference:")
        print(f"   📊 {latest_model['name']}")
        print(f"   📁 {latest_model['path']}")
        print(f"   📏 {latest_model['size']:.1f} MB")
        
        return model_files
    else:
        print("❌ No Recognition model files (.pdparams) found!")
        print("   Make sure training completed successfully")
        return []

def create_training_summary():
    """
    สร้างสรุปการเทรน Recognition
    """
    print(f"\\n" + "="*60)
    print(f"📋 RECOGNITION TRAINING SUMMARY")
    print(f"="*60)
    
    # ข้อมูลพื้นฐาน
    print(f"\\n🎯 Training Configuration:")
    print(f"   Target: Text Recognition Only")
    print(f"   Architecture: CRNN (MobileNetV3 + BiLSTM + CTC)")
    print(f"   Config file: {output_config_path}")
    print(f"   Model save directory: {LOCAL_RECOGNITION_MODEL_DIR}")
    
    # ข้อมูลการเทรน
    print(f"\\n📊 Training Data:")
    print(f"   Train annotation: {train_annotation_path}")
    print(f"   Eval annotation: {eval_annotation_path}")
    print(f"   Data format: image_path\\ttext_content")
    
    # GPU information
    print(f"\\n🎮 Hardware:")
    print(f"   GPU available: {paddle.is_compiled_with_cuda()}")
    if paddle.is_compiled_with_cuda():
        print(f"   GPU count: {paddle.device.cuda.device_count()}")
    
    monitor_gpu_usage()
    
    # ผลลัพธ์
    print(f"\\n🎯 Results:")
    model_files = verify_recognition_model_output()
    
    # S3 sync status
    print(f"\\n☁️  S3 Storage:")
    print(f"   Bucket: s3://{S3_BUCKET}")
    print(f"   Recognition data: {S3_RECOGNITION_DATA_PREFIX}/")
    print(f"   Recognition models: {S3_RECOGNITION_MODEL_PREFIX}/")
    
    # Training logs
    log_file = os.path.join(LOCAL_RECOGNITION_MODEL_DIR, "training.log")
    if os.path.exists(log_file):
        print(f"\\n📝 Training Log Analysis:")
        analyze_training_logs(log_file)
    
    # แนะนำขั้นตอนถัดไป
    print(f"\\n🚀 Next Steps:")
    if model_files:
        print(f"   1. Use the latest .pdparams file for inference")
        print(f"   2. Download model from S3 for deployment")
        print(f"   3. Test recognition with sample images")
        print(f"   4. Deploy model using PaddleOCR inference tools")
    else:
        print(f"   1. Check training logs for errors")
        print(f"   2. Verify annotation file format")
        print(f"   3. Restart training if necessary")
    
    print(f"\\n" + "="*60)

# รันการติดตามและสรุปผล
print("🔍 Starting final monitoring and summary...")

# ติดตาม GPU usage
monitor_gpu_usage()

# ตรวจสอบ training progress
monitor_recognition_training_progress(LOCAL_RECOGNITION_MODEL_DIR)

# สร้างสรุปการเทรน
create_training_summary()

print("\\n✅ Recognition training notebook completed!")
print("\\n💡 Remember:")
print("   - This notebook focuses on Text Recognition only")
print("   - Output files are .pdparams Recognition models")
print("   - Use tools/train_rec.py or tools/train.py for actual training")
print("   - Check docs/problem-log.md before troubleshooting")

## 11. Test Recognition Model (Optional)

ทดสอบ Recognition model ที่เทรนเสร็จแล้วด้วยรูปภาพตัวอย่าง

In [ ]:
def test_recognition_model_inference(model_path, test_image_path, config_path):
    """
    ทดสอบ Recognition model ที่เทรนแล้วด้วยรูปภาพตัวอย่าง
    """
    print(f"🧪 Testing Recognition model inference...")
    print(f"   Model: {model_path}")
    print(f"   Test image: {test_image_path}")
    print(f"   Config: {config_path}")
    
    if not os.path.exists(model_path):
        print(f"❌ Model file not found: {model_path}")
        return False
    
    if not os.path.exists(test_image_path):
        print(f"❌ Test image not found: {test_image_path}")
        return False
    
    try:
        # เตรียมคำสั่งสำหรับ inference
        inference_cmd = [
            "python", "tools/infer_rec.py",
            "-c", config_path,
            "-o", f"Global.pretrained_model={model_path}",
            "-o", f"Global.infer_img={test_image_path}"
        ]
        
        print(f"🚀 Running recognition inference...")
        print(f"   Command: {' '.join(inference_cmd)}")
        
        # รันคำสั่ง inference
        result = subprocess.run(
            inference_cmd,
            cwd=PADDLEOCR_DIR,
            capture_output=True,
            text=True,
            timeout=300
        )
        
        if result.returncode == 0:
            print(f"✅ Inference completed successfully!")
            print(f"📊 Recognition results:")
            
            # แสดงผลลัพธ์
            output_lines = result.stdout.strip().split('\n')
            for line in output_lines[-10:]:  # แสดง 10 บรรทัดสุดท้าย
                if line.strip():
                    print(f"   {line}")
            
            return True
        else:
            print(f"❌ Inference failed!")
            print(f"Error: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"⏰ Inference timed out!")
        return False
    except Exception as e:
        print(f"❌ Error during inference: {e}")
        return False

def create_sample_test_images():
    """
    สร้างรูปภาพตัวอย่างสำหรับทดสอบ Recognition
    """
    print(f"🎨 Creating sample test images for recognition...")
    
    from PIL import Image, ImageDraw, ImageFont
    import numpy as np
    
    # สร้างโฟลเดอร์สำหรับ test images
    test_dir = os.path.join(LOCAL_RECOGNITION_DATA_DIR, "test_samples")
    os.makedirs(test_dir, exist_ok=True)
    
    # ข้อความตัวอย่างสำหรับทดสอบ
    test_texts = [
        "Hello World",
        "PaddleOCR",
        "Recognition Test",
        "สวัสดี",
        "1234567890",
        "ABC123"
    ]
    
    created_images = []
    
    for i, text in enumerate(test_texts):
        # สร้างรูปภาพขาว
        img_width, img_height = 200, 50
        img = Image.new('RGB', (img_width, img_height), color='white')
        draw = ImageDraw.Draw(img)
        
        # เขียนข้อความ
        try:
            # ใช้ font เริ่มต้น
            font_size = 24
            bbox = draw.textbbox((0, 0), text)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]
            
            # คำนวณตำแหน่งกลาง
            x = (img_width - text_width) // 2
            y = (img_height - text_height) // 2
            
            draw.text((x, y), text, fill='black')
            
        except Exception as e:
            print(f"⚠️  Could not draw text '{text}': {e}")
            continue
        
        # บันทึกรูปภาพ
        img_path = os.path.join(test_dir, f"test_{i+1:02d}.jpg")
        img.save(img_path)
        created_images.append({
            'path': img_path,
            'text': text,
            'filename': f"test_{i+1:02d}.jpg"
        })
        
        print(f"   ✓ Created: {img_path} -> '{text}'")
    
    print(f"✅ Created {len(created_images)} test images in: {test_dir}")
    return created_images, test_dir

def run_comprehensive_recognition_test():
    """
    รัน comprehensive test สำหรับ Recognition model
    """
    print(f"🧪 Running comprehensive Recognition model test...")
    
    # หา model file ที่ใหม่ที่สุด
    model_files = []
    for root, dirs, files in os.walk(LOCAL_RECOGNITION_MODEL_DIR):
        for file in files:
            if file.endswith('.pdparams'):
                full_path = os.path.join(root, file)
                model_files.append({
                    'path': full_path,
                    'name': file,
                    'mtime': os.path.getmtime(full_path)
                })
    
    if not model_files:
        print(f"❌ No Recognition model files found!")
        return False
    
    # เลือก model ที่ใหม่ที่สุด
    latest_model = max(model_files, key=lambda x: x['mtime'])
    model_path = latest_model['path'].replace('.pdparams', '')  # ไม่ต้องมี extension
    
    print(f"🎯 Using model: {latest_model['name']}")
    
    # สร้างรูปภาพทดสอบ
    test_images, test_dir = create_sample_test_images()
    
    if not test_images:
        print(f"❌ No test images created!")
        return False
    
    # ทดสอบกับแต่ละรูปภาพ
    success_count = 0
    for test_img in test_images[:3]:  # ทดสอบ 3 รูปแรก
        print(f"\\n🔍 Testing with: {test_img['filename']} (expected: '{test_img['text']}')")
        
        success = test_recognition_model_inference(
            model_path=model_path,
            test_image_path=test_img['path'],
            config_path=output_config_path
        )
        
        if success:
            success_count += 1
    
    print(f"\\n📊 Test Results:")
    print(f"   Tests run: {min(len(test_images), 3)}")
    print(f"   Successful: {success_count}")
    print(f"   Success rate: {(success_count / min(len(test_images), 3)) * 100:.1f}%")
    
    return success_count > 0

# เรียกใช้การทดสอบ (optional)
print("🧪 Recognition Model Testing Section")
print("   This section allows you to test your trained Recognition model")
print("   Uncomment the following lines to run tests:")
print("   # test_success = run_comprehensive_recognition_test()")

# สร้างคำแนะนำสำหรับการใช้งาน model
def create_inference_guide():
    """
    สร้างคำแนะนำการใช้งาน Recognition model
    """
    print(f"\\n📖 RECOGNITION MODEL INFERENCE GUIDE")
    print(f"=" * 50)
    
    # หา model ที่แนะนำ
    model_files = []
    for root, dirs, files in os.walk(LOCAL_RECOGNITION_MODEL_DIR):
        for file in files:
            if file.endswith('.pdparams'):
                full_path = os.path.join(root, file)
                model_files.append({
                    'path': full_path,
                    'name': file,
                    'size_mb': os.path.getsize(full_path) / (1024 * 1024)
                })
    
    if model_files:
        recommended_model = max(model_files, key=lambda x: os.path.getmtime(x['path']))
        
        print(f"\\n🎯 Recommended Model:")
        print(f"   File: {recommended_model['name']}")
        print(f"   Path: {recommended_model['path']}")
        print(f"   Size: {recommended_model['size_mb']:.1f} MB")
        
        print(f"\\n💻 Command Line Usage:")
        model_path_no_ext = recommended_model['path'].replace('.pdparams', '')
        print(f"   cd {PADDLEOCR_DIR}")
        print(f"   python tools/infer_rec.py \\\\")
        print(f"     -c {output_config_path} \\\\")
        print(f"     -o Global.pretrained_model={model_path_no_ext} \\\\")
        print(f"     -o Global.infer_img=path/to/your/image.jpg")
        
        print(f"\\n🐍 Python API Usage:")
        print(f"   from paddleocr import PaddleOCR")
        print(f"   ocr = PaddleOCR(")
        print(f"       use_angle_cls=False,")
        print(f"       lang='en',  # or 'ch' for Chinese")
        print(f"       rec_model_dir='{os.path.dirname(model_path_no_ext)}'")
        print(f"   )")
        print(f"   result = ocr.ocr('path/to/your/image.jpg', det=False)")
        
        print(f"\\n☁️  S3 Model Location:")
        print(f"   s3://{S3_BUCKET}/{S3_RECOGNITION_MODEL_PREFIX}/")
        
        print(f"\\n📋 Model Deployment Tips:")
        print(f"   1. Download .pdparams file from S3")
        print(f"   2. Use with PaddleOCR inference tools")
        print(f"   3. For production, consider model optimization")
        print(f"   4. Test with various image types and sizes")
        
    else:
        print(f"\\n❌ No model files found for inference guide")

create_inference_guide()

## 🎯 Final Summary and Next Steps

เสร็จสิ้นการเตรียมระบบสำหรับ PaddleOCR Recognition Training บน SageMaker

In [ ]:
print("🎯" + "="*70)
print("  PADDLEOCR RECOGNITION TRAINING - FINAL SUMMARY")
print("="*77)

print(f"\n✅ COMPLETED SETUP:")
print(f"   📋 Environment verification and GPU check")
print(f"   📦 PaddleOCR repository cloned and configured")
print(f"   ☁️  S3 integration setup and tested")
print(f"   📝 Recognition annotation format validation")
print(f"   🔧 Training configuration prepared")
print(f"   📊 Monitoring and checkpoint sync ready")

print(f"\n🎯 RECOGNITION TRAINING FOCUS:")
print(f"   ✓ Text Recognition only (no detection)")
print(f"   ✓ CRNN architecture (MobileNetV3 + BiLSTM + CTC)")
print(f"   ✓ Tab-separated annotation format: image_path\\ttext_content")
print(f"   ✓ Output: .pdparams Recognition model files")
print(f"   ✓ S3 integration for data and model storage")

print(f"\n📁 KEY DIRECTORIES:")
print(f"   📂 PaddleOCR repo: {PADDLEOCR_DIR}")
print(f"   📂 Recognition data: {LOCAL_RECOGNITION_DATA_DIR}")
print(f"   📂 Model output: {LOCAL_RECOGNITION_MODEL_DIR}")
print(f"   📂 Config files: {LOCAL_RECOGNITION_CONFIG_DIR}")

print(f"\n☁️  S3 STORAGE:")
print(f"   🪣 Bucket: s3://{S3_BUCKET}")
print(f"   📄 Recognition data: {S3_RECOGNITION_DATA_PREFIX}/")
print(f"   🎯 Recognition models: {S3_RECOGNITION_MODEL_PREFIX}/")

print(f"\n🚀 TO START TRAINING:")
print(f"   1. Ensure GPU is available: {paddle.is_compiled_with_cuda()}")
print(f"   2. Upload your recognition data to S3")
print(f"   3. Uncomment training code in Section 8")
print(f"   4. Run: training_success = start_recognition_training(output_config_path)")
print(f"   5. Monitor progress and sync checkpoints to S3")

print(f"\n📖 TRAINING COMMAND:")
print(f"   cd {PADDLEOCR_DIR}")
print(f"   python tools/train.py -c {output_config_path}")

print(f"\n📚 DOCUMENTATION:")
print(f"   📋 Data format: docs/data-format.md")
print(f"   🔧 Configuration: docs/configuration-guide.md")
print(f"   🐛 Troubleshooting: docs/troubleshooting.md")
print(f"   📝 Problem log: docs/problem-log.md")

print(f"\n⚠️  IMPORTANT NOTES:")
print(f"   • This notebook is for Recognition training only")
print(f"   • No text detection functionality included")
print(f"   • Training requires GPU for reasonable performance")
print(f"   • Check docs/problem-log.md before troubleshooting")
print(f"   • Backup important models to S3 regularly")

print(f"\n🎯 EXPECTED OUTPUT:")
print(f"   📊 Trained Recognition model (.pdparams)")
print(f"   📈 Training logs and metrics")
print(f"   💾 Model checkpoints in S3")
print(f"   🧪 Ready for inference and deployment")

print(f"\n💡 NEXT STEPS AFTER TRAINING:")
print(f"   1. Download latest .pdparams from S3")
print(f"   2. Test model with sample images")
print(f"   3. Deploy using PaddleOCR inference API")
print(f"   4. Integrate into your application")

print(f"\n🔗 USEFUL COMMANDS:")
print(f"   # Check GPU status")
print(f"   nvidia-smi")
print(f"   ")
print(f"   # Monitor training progress")
print(f"   tail -f {LOCAL_RECOGNITION_MODEL_DIR}/training.log")
print(f"   ")
print(f"   # Test recognition inference")
print(f"   python tools/infer_rec.py -c config.yml -o Global.infer_img=test.jpg")

print(f"\n" + "="*77)
print(f"✅ PaddleOCR Recognition Training setup completed successfully!")
print(f"🚀 Ready to train Text Recognition models on SageMaker")
print(f"="*77)